#### clustering

In [27]:
from itertools import combinations

In [2]:
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/")
in_list = glob.glob("*.txt")
df = pd.read_table(in_list[0])
df["effectSize_%s"% in_list[0].split(".")[0]] = df['foldchange']
for sample in in_list[1:]:
    df_i = pd.read_table(sample)
    df_i["effectSize_%s"% sample.split(".")[0]] = df_i['foldchange']
    df = df.merge(df_i, on=['contig','position','refAllele','altAllele'], how='outer')
df['term'] = df['contig'] + ";" + df['position'].astype(str) + ";" + df['refAllele'] + ";" + df['altAllele']
cols = ['term'] + [x for x in df.columns if "effectSize_" in x]
df = df[cols]
df = df.set_index(['term'])
# 
df_corr = spearman(df)
df_corr.to_csv("sample_correlation.txt", sep="\t")

In [44]:
# ASE
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/by_ind/sig/")
ind_list = glob.glob("ind*.txt")
fetal_list = ["individual_%d.txt"% i for i in range(1,7)]
adult_list = ["individual_%d.txt"% i for i in range(7,14)]

def allele_matrix(in_list):
    df = pd.read_table(in_list[0])
    effect_cols = [x for x in df.columns if "effect_" in x]
    df["median_%s"%(in_list[0].split(".txt")[0])] = df[effect_cols].median(axis=1)
    for ind in in_list[1:]:
        df_i = pd.read_table(ind)
        effect_cols = [x for x in df_i.columns if "effect_" in x]
        df_i["median_%s"%(ind.split(".txt")[0])] = df_i[effect_cols].median(axis=1)
        #
        df = df.merge(df_i, on=['contig','position','refAllele','altAllele'], how='outer')
    df['term'] = df['contig'] + ";" + df['position'].astype(str) + ";" + df['refAllele'] + ";" + df['altAllele']
    cols = ['term'] + [x for x in df.columns if "median_" in x]
    df = df[cols]
    df = df.set_index(['term'])
    return df

df_fetal = allele_matrix(fetal_list)
df_adult = allele_matrix(adult_list)
print(df_fetal.head())

                   median_individual_1         ...           median_individual_6
term                                           ...                              
chr1;1650807;T;C              0.333511         ...                      0.384083
chr1;1650832;A;G              2.095892         ...                      3.000000
chr1;3807258;G;C              0.260390         ...                           NaN
chr1;16798902;G;A             3.462608         ...                           NaN
chr1;23688933;A;G             2.201389         ...                           NaN

[5 rows x 6 columns]


In [1]:
def spearman(df):
    coef_array = []
    col_list = df.columns
    for x in col_list:
        coef_list = []
        for y in col_list: 
            df_i = df[[x, y]].dropna(how="any")
            coef = df_i.corr(method="spearman").iloc[0,1]
            coef_list.append(coef)
        coef_array.append(coef_list)
    df_result = pd.DataFrame(coef_array)
    df_result.columns = col_list
    df_result.index = col_list
    return df_result

# df_corr_fetal = spearman(df_fetal)
# df_corr_fetal.to_csv("fetal_correlation.txt", sep="\t")
# df_corr_adult = spearman(df_adult)
# df_corr_adult.to_csv("adult_correlation.txt", sep="\t")
# print(df_corr_fetal.head())
# df_total = df_fetal.merge(df_adult, how="outer", left_index=True, right_index=True)
# df_corr_total = spearman(df_total)
# df_corr_total.to_csv("total_correlation.txt", sep="\t")
# print(df_total.head())